In [1]:
using LinearAlgebra
using JuMP
using GLPK
using XLSX
using Complementarity #not sure I need this one?
using PATHSolver
using Revise #not having to restart sessions
#using Pandas #optional (only for EXIOBASE data)

include("../src/SUT_structure.jl") # Used for the SUT setup <sut = SUT.structure(...)>
include("../src/Constructs.jl") # Used to derive single-production systems from the SUT setup, e.g. as <itc = Constructs.ITC(sut)>
include("../src/Auxiliary.jl") # Includes some helper functions
include("../src/Model_data.jl") # Sets up the data structure for RCOT modelling based on SUT.structure or Constructs.construct
include("../src/RCOT_model.jl") # Builds and solves the RCOT models
include("../src/LP_models.jl"); # Builds and solves the LP models

In [2]:
# Load the workbook
xf = XLSX.readxlsx("../data/SUT_c+i-.xlsx")

XLSXFile("../data/SUT_c+i-.xlsx") containing 4 Worksheets
            sheetname size          range        
-------------------------------------------------
                    V 4x6           A1:F4        
                  U+e 6x5           A1:E6        
                    F 5x4           A1:D5        
                   pi 5x2           A1:B5        


In [3]:
# Get data from all worksheets and convert it to float64 data type
V = convert(Matrix{Float64}, xf["V!B2:F4"])
U = convert(Matrix{Float64}, xf["U+e!B2:D6"])
e = convert(Matrix{Float64}, xf["U+e!E2:E6"])
F = convert(Matrix{Float64}, xf["F!B2:D5"])
pii = convert(Matrix{Float64}, xf["pi!B2:B5"])
t = xf["V!A1"];

In [4]:
sut = SUT.structure(U,V,F,e,pii,t);

[ Info: A structure for supply-use elements was set up. Changing individual elements will not change others automatically.


In [5]:
itc = Constructs.ITC(sut)

construct([88.0; 95.0; … ; 10.0; 10.0;;], [57.0; 79.0; … ; 4.0; 2.0;;], [8.0 23.0 … 0.0 0.0; 0.0 6.999999999999999 … 0.9375 0.9375; … ; 2.0 4.0 … 0.0 0.0; 0.0 0.0 … 0.8333333333333333 0.8333333333333333], [0.09090909090909091 0.24210526315789474 … 0.0 0.0; 0.0 0.07368421052631578 … 0.09375 0.09375; … ; 0.022727272727272728 0.042105263157894736 … 0.0 0.0; 0.0 0.0 … 0.08333333333333333 0.08333333333333333], [1.1163754045307444 0.2947572815533981 … 0.032750809061488674 0.032750809061488674; 0.061488673139158574 1.1067961165048543 … 0.12297734627831716 0.12297734627831715; … ; 0.027961165048543693 0.053300970873786414 … 1.0059223300970874 0.00592233009708738; 0.05062927004674577 0.022437971952535062 … 0.10125854009349156 1.1012585400934916], [17.5 16.666666666666668 … 0.0 0.0; 1.6666666666666667 6.944444444444445 … 0.7812499999999999 0.7812499999999999; 0.0 0.7407407407407407 … 0.15432098765432098 0.15432098765432098; 6.19047619047619 4.761904761904762 … 2.033730158730158 2.033730158730158

In [6]:
model_data = Model_data.IO(itc)

[ Info: You are setting up an IO model dataset. Elements of this dataset are now treated independently, meaning that no recalculation whatsoever takes place when individual elements are changed.


construct([88.0; 95.0; … ; 10.0; 10.0;;], [57.0; 79.0; … ; 4.0; 2.0;;], [8.0 23.0 … 0.0 0.0; 0.0 6.999999999999999 … 0.9375 0.9375; … ; 2.0 4.0 … 0.0 0.0; 0.0 0.0 … 0.8333333333333333 0.8333333333333333], [0.09090909090909091 0.24210526315789474 … 0.0 0.0; 0.0 0.07368421052631578 … 0.09375 0.09375; … ; 0.022727272727272728 0.042105263157894736 … 0.0 0.0; 0.0 0.0 … 0.08333333333333333 0.08333333333333333], [1.1163754045307444 0.2947572815533981 … 0.032750809061488674 0.032750809061488674; 0.061488673139158574 1.1067961165048543 … 0.12297734627831716 0.12297734627831715; … ; 0.027961165048543693 0.053300970873786414 … 1.0059223300970874 0.00592233009708738; 0.05062927004674577 0.022437971952535062 … 0.10125854009349156 1.1012585400934916], [17.5 16.666666666666668 … 0.0 0.0; 1.6666666666666667 6.944444444444445 … 0.7812499999999999 0.7812499999999999; 0.0 0.7407407407407407 … 0.15432098765432098 0.15432098765432098; 6.19047619047619 4.761904761904762 … 2.033730158730158 2.033730158730158

In [7]:
model_data.I_mod

5×5 Matrix{Bool}:
 1  0  0  0  0
 0  1  0  0  0
 0  0  1  0  0
 0  0  0  1  0
 0  0  0  0  1

In [10]:
model_data.x

5×1 Matrix{Float64}:
 88.0
 95.0
 76.0
 10.0
 10.0

In [8]:
io_dosso("primal", "rel", model_data);#, surplus=false);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 169.0
value.(var_con) = [88.0, 95.0, 76.00000000000001, 10.0, 10.0]
value.(demand_con) = [57.0, 79.0, 27.0, 4.0, 2.0]
value.(factor_con) = Float64[]


In [9]:
io_dosso("dual", "abs", model_data);#, surplus=false);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 169.0
value.(p) = [1.0, 1.0, 1.0000000000000002, 1.0, 0.9999999999999998]
value.(r) = Float64[]
value.(profit_con) = [40.0, 57.0, 57.0, 7.499999999999998, 7.499999999999998]


In [15]:
#Flipped objective and constraints for io_dosso
# to check if they yield the same result... still need to check for differing y etc.

# Instantiate the model
primal = Model(GLPK.Optimizer)
(@isdefined primal) && (primal = nothing; primal = Model(GLPK.Optimizer));
# Define the optimisation model
@variable(primal, x_star[1:size(itc.A,1)] >= 0, base_name = "x*")
@variable(primal, p_star[1:size(itc.A,2)] >= 0, base_name = "p*")
@objective(primal, Max, sum(model_data.pii'*model_data.R*x_star))
@constraint(primal, c1, (model_data.I_mod - model_data.A)*x_star .<= model_data.y);
# Run the model and show results
optimize!(primal)
@show termination_status(primal)
@show primal_status(primal)
@show dual_status(primal)
@show objective_value(primal)
@show value.(x_star)
@show value.(p_star)
@show value.(c1)
@show shadow_price.(c1)

termination_status(primal) = MathOptInterface.OPTIMAL
primal_status(primal) = MathOptInterface.FEASIBLE_POINT
dual_status(primal) = MathOptInterface.FEASIBLE_POINT
objective_value(primal) = 169.0
value.(x_star) = [88.0, 95.0, 76.00000000000001, 10.0, 10.0]
value.(p_star) = [0.0, 0.0, 0.0, 0.0, 0.0]
value.(c1) = [57.0; 79.0; 27.0; 4.0; 2.0;;]
shadow_price.(c1) = [1.0; 1.0; 1.0; 1.0; 1.0;;]


5×1 Matrix{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0

In [11]:
# DOSSO - MCP
dosso_mcp = Model(PATHSolver.Optimizer)

act = size(model_data.A,1)
com = size(model_data.A,2)

# Define the optimisation model
@variable(dosso_mcp, x_star[1:act] >= 0)
@variable(dosso_mcp, p_star[1:com] >= 0)

@constraint(dosso_mcp, c1, vec((model_data.pii)'*model_data.R - (p_star'*(model_data.I_mod - model_data.A))) ⟂ x_star)
@constraint(dosso_mcp, c2, vec((model_data.I_mod - model_data.A)*(x_star) - (model_data.y)) ⟂ p_star)

# Solve
optimize!(dosso_mcp);

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris


Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             2.0238e+02             0.0e+00 (f[    7])
    1     1     8     5 1.7837e+02  1.0e+00    2.0e+00 (f[    7])
    2     2     3     8 1.4482e+02  1.0e+00    1.8e+00 (f[    2])
    3     3     0     8 1.1975e+02  1.0e+00    1.4e+00 (f[    2])
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     4     4 1.1975e+02           I 1.2e+00 7.8e+01 (f[    2])
    1     3     5     5 5.5178e+01  1.0e+00 SO 4.8e-01 3.2e+01 (f[    3])
    2     2     6     6 2.6727e+00  1.0e+00 SO 1.9e-01 1.2e+00 (f[    8])
    3     3     7     7 1.1607e+00  1.0e+00 SO 7.7e-02 7.9e-01 (f[    3])
    4     4     8     8 4.5177e-01  1.0e+00 SO 3.1e-02 3.2e-01 (f[    2])
    5     1     9     9 8.2929e-03  1.0e+00 SO 1.

In [12]:
solution_summary(dosso_mcp; verbose = true)

* Solver : Path 5.0.03

* Status
  Result count       : 1
  Termination status : LOCALLY_SOLVED
  Message from the solver:
  "The problem was solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 0.00000e+00
  Primal solution :
    p_star[1] : 1.00000e+00
    p_star[2] : 1.00000e+00
    p_star[3] : 1.00000e+00
    p_star[4] : 1.00000e+00
    p_star[5] : 1.00000e+00
    x_star[1] : 8.80000e+01
    x_star[2] : 9.50000e+01
    x_star[3] : 7.60000e+01
    x_star[4] : 1.00000e+01
    x_star[5] : 1.00000e+01

* Work counters
  Solve time (sec)   : 3.10000e-02


In [26]:
dosso_primal = Model(GLPK.Optimizer)
(@isdefined dosso_primal) && (dosso_primal = nothing; dosso_primal = Model(GLPK.Optimizer));

@variable(dosso_primal, x_star[1:size(model_data.A,2)] >= 0, base_name = "x*")
@objective(dosso_primal, Min, sum(model_data.pii'*model_data.R*x_star))
@constraint(dosso_primal, mass_balance, (model_data.I_mod-model_data.A)*(x_star) .>= model_data.y.+1)
optimize!(dosso_primal)
model_solution(dosso_primal)
show_primal_lhs(dosso_primal)

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 174.0
value.(var_con) = [89.5093851132686, 96.53721682847896, 78.04624235886372, 11.099029126213592, 11.376842862279755]
value.(demand_con) = [58.0, 80.0, 28.0, 5.0, 3.0]
value.(factor_con) = Float64[]


Float64[]

In [33]:
dosso_dual = Model(GLPK.Optimizer)
(@isdefined dosso_dual) && (dosso_dual = nothing; dosso_dual = Model(GLPK.Optimizer));

@variable(dosso_dual, p[1:size(model_data.A,2)] >= 0)
@objective(dosso_dual, Max, sum(p'*(model_data.y.+1)))
@constraint(dosso_dual, factor_costs, (model_data.I_mod-model_data.A)'*(p) .<= model_data.R'*model_data.pii)
optimize!(dosso_dual)
model_solution(dosso_dual)
show_dual_lhs(dosso_dual)

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 174.0
value.(p) = [1.0, 1.0000000000000002, 1.0, 1.0, 1.0]
value.(r) = Float64[]
value.(profit_con) = [0.45454545454545453, 0.6000000000000001, 0.75, 0.75, 0.75]


5-element Vector{Float64}:
 0.45454545454545453
 0.6000000000000001
 0.75
 0.75
 0.75

In [63]:
##########
#Ghosh

B_ghosh = inv(Diagonal(vec(model_data.x)))*model_data.Z
D_ghosh = inv(Diagonal(vec(model_data.x)))*(model_data.y)
π_ghosh = 1 #only scalar because single demand category
v_ghosh = (model_data.pii'*model_data.G)' # transpose necessary to make column vector


5×1 adjoint(::Matrix{Float64}) with eltype Float64:
 40.0
 57.0
 57.0
  7.499999999999998
  7.499999999999998

In [72]:
# DOSSO - MCP
# for GHOSH
ghosh_mcp = Model(PATHSolver.Optimizer)

act = size(model_data.A,1)
com = size(model_data.A,2)

# Define the optimisation model
@variable(ghosh_mcp, x_star[1:act] >= 0)
@variable(ghosh_mcp, p_star[1:com] >= 0)

@constraint(ghosh_mcp, c1, vec((D_ghosh*π_ghosh)' - p_star'*(model_data.I_mod-B_ghosh)') ⟂ x_star)
@constraint(ghosh_mcp, c2, vec((model_data.I_mod - B_ghosh)'*(x_star) - (v_ghosh)) ⟂ p_star)

# Solve
optimize!(ghosh_mcp)

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris


Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             1.8122e+02             0.0e+00 (f[    7])
    1     1     8     5 1.5801e+02  1.0e+00    1.8e+00 (f[    7])
    2     2     3     8 1.2665e+02  1.0e+00    1.6e+00 (f[    2])
    3     3     0     8 1.0320e+02  1.0e+00    1.3e+00 (f[    2])
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     4     4 1.0320e+02           I 1.0e+00 6.6e+01 (f[    2])
    1     3     5     5 4.4453e+01  1.0e+00 SO 4.1e-01 2.7e+01 (f[    1])
    2     2     6     6 3.6138e+00  1.0e+00 SO 1.7e-01 2.1e+00 (f[    5])
    3     4     7     7 1.1998e+00  1.0e+00 SO 6.6e-02 8.4e-01 (f[    2])
    4     5     8     8 2.1283e-01  1.0e+00 SO 2.6e-02 1.7e-01 (f[    3])
    5     1     9     9 4.1240e-03  1.0e+00 SO 1.

In [73]:
solution_summary(ghosh_mcp; verbose = true)

* Solver : Path 5.0.03

* Status
  Result count       : 1
  Termination status : LOCALLY_SOLVED
  Message from the solver:
  "The problem was solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 0.00000e+00
  Primal solution :
    p_star[1] : 1.01715e+00
    p_star[2] : 1.01618e+00
    p_star[3] : 1.02692e+00
    p_star[4] : 1.10990e+00
    p_star[5] : 1.13768e+00
    x_star[1] : 8.80000e+01
    x_star[2] : 9.50000e+01
    x_star[3] : 7.60000e+01
    x_star[4] : 1.00000e+01
    x_star[5] : 1.00000e+01

* Work counters
  Solve time (sec)   : 0.00000e+00


In [53]:
# DOSSO - MCP
# for GHOSH
ghosh_mcp = Model(PATHSolver.Optimizer)

act = size(model_data.A,1)
com = size(model_data.A,2)

# Define the optimisation model
@variable(ghosh_mcp, x_star[1:act] >= 0)
@variable(ghosh_mcp, p_star[1:com] >= 0)

@constraint(ghosh_mcp, c1, vec((D_ghosh*π_ghosh) - (model_data.I_mod-B_ghosh)*p_star) ⟂ x_star)
@constraint(ghosh_mcp, c2, vec((model_data.I_mod - B_ghosh)'*(x_star) - (v_ghosh)) ⟂ p_star)

# Solve
optimize!(ghosh_mcp)

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris


Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             1.8122e+02             0.0e+00 (f[    7])
    1     1     8     5 1.5802e+02  1.0e+00    1.8e+00 (f[    7])
    2     2     3     8 1.2666e+02  1.0e+00    1.6e+00 (f[    2])
    3     3     0     8 1.0321e+02  1.0e+00    1.3e+00 (f[    2])
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     4     4 1.0321e+02           I 1.0e+00 6.6e+01 (f[    2])
    1     3     5     5 4.4466e+01  1.0e+00 SO 4.1e-01 2.7e+01 (f[    1])
    2     2     6     6 3.5426e+00  1.0e+00 SO 1.7e-01 2.1e+00 (f[    5])
    3     4     7     7 1.1556e+00  1.0e+00 SO 6.6e-02 8.3e-01 (f[    2])
    4     5     8     8 2.2848e-01  1.0e+00 SO 2.6e-02 1.8e-01 (f[    3])
    5     1     9     9 4.5997e-03  1.0e+00 SO 1.

In [54]:
solution_summary(ghosh_mcp; verbose = true)

* Solver : Path 5.0.03

* Status
  Result count       : 1
  Termination status : LOCALLY_SOLVED
  Message from the solver:
  "The problem was solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 0.00000e+00
  Primal solution :
    p_star[1] : 1.00000e+00
    p_star[2] : 1.00000e+00
    p_star[3] : 1.00000e+00
    p_star[4] : 1.00000e+00
    p_star[5] : 1.00000e+00
    x_star[1] : 8.80000e+01
    x_star[2] : 9.50000e+01
    x_star[3] : 7.60000e+01
    x_star[4] : 1.00000e+01
    x_star[5] : 1.00000e+01

* Work counters
  Solve time (sec)   : 0.00000e+00


In [70]:
#####
# DOSSO for Ghosh
# I used the transposed system
ghosh_primal = Model(GLPK.Optimizer)
(@isdefined ghosh_primal) && (ghosh_primal = nothing; ghosh_primal = Model(GLPK.Optimizer));

@variable(ghosh_primal, x_star[1:size(model_data.A,2)] >= 0, base_name = "x*")
@objective(ghosh_primal, Min, sum((D_ghosh*π_ghosh)'*x_star))
@constraint(ghosh_primal, mass_balance, (model_data.I_mod-B_ghosh)'*(x_star) .>= v_ghosh)
optimize!(ghosh_primal)
model_solution(ghosh_primal)
show_primal_lhs(ghosh_primal)

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 174.0
value.(var_con) = [88.0, 95.0, 76.0, 9.999999999999998, 10.0]
value.(demand_con) = [40.0, 57.0, 57.0, 7.499999999999998, 7.499999999999998]
value.(factor_con) = Float64[]


Float64[]

In [71]:
value.(x_star)

5-element Vector{Float64}:
 88.0
 95.0
 76.0
  9.999999999999998
 10.0

In [68]:
##########
#Ghosh

B_ghosh = inv(Diagonal(vec(model_data.x)))*model_data.Z
D_ghosh = inv(Diagonal(vec(model_data.x)))*(model_data.y .+1)
π_ghosh = 1 #only scalar because single demand category
v_ghosh = (model_data.pii'*model_data.G)' # transpose necessary to make column vector


5×1 adjoint(::Matrix{Float64}) with eltype Float64:
 40.0
 57.0
 57.0
  7.499999999999998
  7.499999999999998

In [69]:
#####
# DOSSO for Ghosh
ghosh_dual = Model(GLPK.Optimizer)
(@isdefined ghosh_dual) && (ghosh_dual = nothing; ghosh_dual = Model(GLPK.Optimizer));

@variable(ghosh_dual, p_star[1:size(model_data.A,2)] >= 0, base_name = "p*")
@objective(ghosh_dual, Max, sum(v_ghosh'*p_star))
@constraint(ghosh_dual, mass_balance, (model_data.I_mod-B_ghosh)*(p_star) .<= D_ghosh*π_ghosh)
optimize!(ghosh_dual)
model_solution(ghosh_dual)
show_primal_lhs(ghosh_dual)

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 174.0
value.(var_con) = [1.0171521035598707, 1.0161812297734627, 1.0269242415639963, 1.1099029126213595, 1.1376842862279755]
value.(factor_con) = [0.6590909090909091, 0.8421052631578947, 0.3684210526315789, 0.5, 0.30000000000000004]


5-element Vector{Float64}:
 0.6590909090909091
 0.8421052631578947
 0.3684210526315789
 0.5
 0.30000000000000004

In [74]:
value.(p_star)

5-element Vector{Float64}:
 1.0171521035594615
 1.0161812297731443
 1.0269242415634634
 1.1099029126210394
 1.1376842862274097

In [78]:
value.(x_star)

5-element Vector{Float64}:
 89.50938511329217
 96.53721682849903
 78.04624235888987
 11.099029126219682
 11.376842862288468

In [75]:
value.(p_star).*model_data.x

5×1 Matrix{Float64}:
 89.50938511323261
 96.5372168284487
 78.04624235882322
 11.099029126210393
 11.376842862274097

In [80]:
net_out = model_data.I_mod-model_data.A
output = inv(net_out)*(model_data.y .+1)

5×1 Matrix{Float64}:
 89.5093851132686
 96.53721682847898
 78.04624235886372
 11.099029126213594
 11.376842862279755

# DOSSO-Ghosh LCP

In [ ]:
##########
#Ghosh

B_ghosh = inv(Diagonal(vec(model_data.x)))*model_data.Z
D_ghosh = inv(Diagonal(vec(model_data.x)))*(model_data.y .+1)
π_ghosh = 1 #only scalar because single demand category
v_ghosh = (model_data.pii'*model_data.G)' # transpose necessary to make column vector


5×1 adjoint(::Matrix{Float64}) with eltype Float64:
 40.0
 57.0
 57.0
  7.499999999999998
  7.499999999999998

In [ ]:
# IO-Ghosh - DOSSO - MCP
# To set up our initial primal, we write:
compl = Model(PATHSolver.Optimizer)

act = size(model_data.A,1)
com = size(model_data.A,2)

# Define the optimisation model
@variable(compl, x_star[1:act] >= 0)
@variable(compl, p_star[1:com] >= 0)
@variable(compl, pii_star[1:fac] >= 0)

@constraint(compl, c1, vec(D_ghosh*π_ghosh - (model_data.I_mod - B_ghosh)*p_star) ⟂ x_star)
@constraint(compl, c2, vec((x_star)'*(model_data.I_mod - B_ghosh) - v_ghosh) ⟂ p_star)

# Solve
optimize!(compl)

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris


Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             1.4596e+02             0.0e+00 (f[    4])
    1     1     6     3 1.1819e+02  1.0e+00    1.5e+00 (f[    4])
    2     2     3     6 9.5511e+01  1.0e+00    1.2e+00 (f[    6])
    3     3     0     6 8.2900e+01  1.0e+00    9.6e-01 (f[    3])
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     4     4 8.2900e+01           I 8.3e-01 4.1e+01 (f[    3])
    1     1     5     5 5.5202e+01  1.0e+00 SO 3.3e-01 4.2e+01 (f[    3])
    2     5     6     6 1.6822e+01  1.0e+00 SO 1.3e-01 7.4e+00 (f[    6])
    3     1     7     7 2.7560e+00  1.0e+00 SO 5.3e-02 1.7e+00 (f[    3])
    4     1     8     8 1.7326e-01  1.0e+00 SO 2.1e-02 7.9e-02 (f[    3])
    5     1     9     9 4.3167e-03  1.0e+00 SO 8.

In [6]:
# NST - MCP

model_data = sut
# To set up our initial primal, we write:
compl = Model(PATHSolver.Optimizer)

act = size(model_data.V,1)
com = size(model_data.V,2)
fac = size(model_data.F,1)

# Define the optimisation model
@variable(compl, z_star[1:act] >= 0)
@variable(compl, p_star[1:com] >= 0)
@variable(compl, pii_star[1:fac] >= 0)

@constraint(compl, c1, vec(pii_star'*model_data.F - (p_star'*(model_data.V' - model_data.U))) ⟂ z_star)
@constraint(compl, c2, vec((model_data.V' - model_data.U)*(z_star) - model_data.e) ⟂ p_star)
@constraint(compl, c3, vec((model_data.ϕ - model_data.F*z_star)) ⟂ pii_star)

# Solve
optimize!(compl)

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Zero:     0 Single:     1 Double:     0
Preprocessed size   : 11

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             2.0970e+02             0.0e+00 (f[    5])
    1    11     4     2 2.0969e+02  3.2e-04    2.1e+00 (f[    5])
    2    12     2     4 4.2741e+00  1.0e+00    1.9e+00 (f[    2])
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0    13     3 4.2741e+00           I 4.3e-02 3.0e+00 (f[    2])
    1     8    14     4 5.4861e-04  1.0e+00 SO 1.7e-02 3.8e-04 (f[    5])
    2     1    15     5 4.0619e-10  1.0e+00 SO 5.5e-05 3.0e-10 (f[    1])

Major Iterations. . . . 2
Minor Iterations. . . . 9
Restarts. . . . . . . . 0
Crash Iterations. . . . 2
Gradient Steps. . . . . 0
Function Evaluations. . 15
Gradient Evaluations. . 5
Basis Time. . 

In [7]:
solution_summary(compl; verbose = true)

* Solver : Path 5.0.03

* Status
  Result count       : 1
  Termination status : LOCALLY_SOLVED
  Message from the solver:
  "The problem was solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 0.00000e+00
  Primal solution :
    p_star[1] : 4.14990e-03
    p_star[2] : 3.84194e-03
    p_star[3] : 2.69926e-03
    p_star[4] : 2.29207e-04
    p_star[5] : 0.00000e+00
    pii_star[1] : 1.04564e-02
    pii_star[2] : 3.70188e-03
    pii_star[3] : 4.43726e-04
    pii_star[4] : 6.43020e-03
    z_star[1] : 1.00000e+00
    z_star[2] : 1.00000e+00
    z_star[3] : 1.00000e+00

* Work counters
  Solve time (sec)   : 1.41000e-01


In [8]:
# SU-NEO - LP
# Instantiate the model
primal = Model(GLPK.Optimizer)
(@isdefined primal) && (primal = nothing; primal = Model(GLPK.Optimizer));
# Define the optimisation model
@variable(primal, z_star[1:size(V,1)] >= 0, base_name = "z*")
@variable(primal, c_star >= 0, base_name = "c*")
@objective(primal, Max, c_star)
@constraint(primal, c1, (model_data.V'-model_data.U)*(z_star) .>= c_star*model_data.e)
@constraint(primal, c2, model_data.F*z_star .<= model_data.ϕ);
# Run the model and show results
optimize!(primal)
@show termination_status(primal)
@show primal_status(primal)
@show dual_status(primal)
@show objective_value(primal)
@show value.(z_star)
@show value.(c_star)
@show value.(c1)
@show value.(c2)
@show shadow_price.(c1)
@show shadow_price.(c2)

termination_status(primal) = MathOptInterface.OPTIMAL
primal_status(primal) = MathOptInterface.FEASIBLE_POINT
dual_status(primal) = MathOptInterface.FEASIBLE_POINT
objective_value(primal) = 1.0000000000000002
value.(z_star) = [1.0000000000000002, 1.0000000000000002, 1.0000000000000002]
value.(c_star) = 1.0000000000000002
value.(c1) = [0.0; 0.0; 0.0; 0.0; 0.0;;]
value.(c2) = [34.16666666666667; 16.111111111111114; 2.2222222222222223; 30.47619047619048;;]
shadow_price.(c1) = [0.0068596694505640736; 0.007379859510174129; 0.000962590370892278; -0.0; -0.0;;]
shadow_price.(c2) = [0.02926829268292683; 0.0; 0.0; 0.0;;]


4×1 Matrix{Float64}:
 0.02926829268292683
 0.0
 0.0
 0.0

In [9]:
#SU-NEO - LP
# Instantiate the model
dual = Model(GLPK.Optimizer)
(@isdefined dual) && (dual = nothing; dual = Model(GLPK.Optimizer));

# Set up the model
dual = Model(GLPK.Optimizer)

# Define the optimisation model
@variable(dual, p[1:size(sut.V,2)] >= 0)
@variable(dual, pii[1:size(sut.F,1)] >= 0)
@objective(dual, Min, sum(pii'*model_data.ϕ))
@constraint(dual, c1d, (model_data.V'-model_data.U)'*(p) .<= model_data.F'*pii)
@constraint(dual, c2d, (p'*model_data.e) .== sum(model_data.e))# or .==1?

# Run the model and show results
optimize!(dual)
model_solution(dual)
@show value.(p)
@show value.(pii)
@show shadow_price.(c1d)
@show shadow_price.(c2d);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 169.0
value.(p) = [1.1592841371453286, 1.2471962572194275, 0.16267777268079495, 0.0, 0.0]
value.(pii) = [4.946341463414633, 0.0, 0.0, 0.0]
shadow_price.(c1d) = [-1.0, -1.0, -1.0000000000000002]
shadow_price.(c2d) = [-1.0;;]
